In [1]:
CPU_CORES = 51
addprocs(CPU_CORES - 1);
@everywhere using DistributedArrays, JuMP, Distributions, Ipopt
using StatsBase

midpoints(AbstractArray{T, 1} where T) in module Base at deprecated.jl:56 overwritten in module StatsBase at /home/cqgz21/.julia/v0.6/StatsBase/src/hist.jl:533.


midpoints(Base.Range{T} where T) in module Base at deprecated.jl:56 overwritten in module StatsBase at /home/cqgz21/.julia/v0.6/StatsBase/src/hist.jl:535.
midpoints(Base.Range{T} where T) in module Base at deprecated.jl:56 overwritten in module StatsBase at /home/cqgz21/.julia/v0.6/StatsBase/src/hist.jl:535.


In [2]:
print(nprocs(),"\n")
print(nworkers(),"\n")
print(workers(),"\n")

51
50
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51]


In [3]:
@everywhere landscape1= [0. 0 0 2 2 0 0 0 2 0 0 2;
              2 0 1 2 5 5 0 0 2 0 0 0;
              3 1 2 2 1 1 2 0 0 3 0 0;
              2 1 1 2 1 2 2 0 3 4 4 0
              2 2 1 1 1 1 1 0 3 4 4 3;
              2 2 1 1 1 1 1 1 3 3 3 0;
              0 2 0 1 5 5 5 0 1 1 0 3;
              3 0 0 1 0 0 2 0 0 3 1 3;
              3 4 0 1 1 3 2 2 2 3 3 3;
              4 3 2 0 0 3 2 2 2 3 2 2] + 1
@everywhere Ncell = 30

In [4]:
@everywhere function create_pixel_mat(landscape, ordered_vec)
    retmat = deepcopy(landscape)
    for i = 1:length(ordered_vec)
        retmat[abs.(retmat - i) .< 0.000000001] = ordered_vec[i]
    end
    
    if size(retmat) == (10,12)
        retmat_zone = hcat(reshape(retmat[1:5 , 1:6],Ncell),
            reshape(retmat[1:5 , 7:12],Ncell),
            reshape(retmat[6:10 , 1:6],Ncell),
            reshape(retmat[6:10 , 7:12],Ncell))
    end
    retmat_zone
end


In [5]:
# Using the prior and visibility info given in the DCA paper
# These have @everywhere because they are fixed. Better declare them in each process rather than
# copying from the parent process
@everywhere s1 = create_pixel_mat(landscape1, [0.4,0.9,0.3,0.2,0.1,0.8])
@everywhere s2 = create_pixel_mat(landscape1, [0.5,0.1,0.1,0.7,0.6,0.9])
@everywhere s3 = create_pixel_mat(landscape1, [0.6,0.1,0.4,0.8,0.7,0.1])
@everywhere s4 = create_pixel_mat(landscape1, [0.8,0.1,0.6,0.2,0.1,0.7])
@everywhere s5 = create_pixel_mat(landscape1, [0.5,0.3,0.5,0.4,0.3,0.6])
@everywhere s6 = create_pixel_mat(landscape1, [0.1,0.5,0.2,0.6,0.5,0.2])




@everywhere Z = 4 # number of zones
@everywhere S = 6 # number of sensors
@everywhere W = reshape(hcat(s1,s2,s3,s4,s5,s6),Ncell,Z,S) # visibility tensor



@everywhere θ = 0.3
@everywhere N = 50
@everywhere Threshold = convert(Int64,floor(N * θ)) # for accepting P computed from each sample
@everywhere epsilon = 1e-5


# No @everywhere because these are only used by the parent process
maxiter = 15 # number of iterations for 



# No @everywhere because these are changing, pass into functions as parameters

Φ = 5
prior_ordered_vec = [0.0085,0.001,0.0115,0.013,0.014,0] # prior of target, order given in the paper
prior = create_pixel_mat(landscape1, prior_ordered_vec)
print(sum(prior))





1.0000000000000004

In [6]:



####################################
## CE Algorithm parallel N

@everywhere function solve_alloc(u, Φ,prior)

    m_ϕ = Model(solver = IpoptSolver(print_level=0))

    @variable(m_ϕ,ϕvar[1:Ncell,1:Z,1:S]) # resource allocation for each senor in each zone in each cell
    @variable(m_ϕ,exponential[1:Ncell,1:Z])
    @variable(m_ϕ, aux[1:Ncell,1:Z])
    @objective(m_ϕ, Min, sum(prior .* exponential))
    @NLconstraint(m_ϕ, [i=1:Ncell,j=1:Z], exp(aux[i,j]) <= exponential[i,j])
    @constraint(m_ϕ,aux .== squeeze(-sum(W .* ϕvar .* reshape(u,1,Z,S),3),3))
    @constraint(m_ϕ, exponential .>= 0)
    @constraint(m_ϕ,ϕvar .>= 0)
    @constraint(m_ϕ,[i=1:Z,j=1:S],sum(ϕvar[:,i,j]) <= Φ)

    solve(m_ϕ)
    getobjectivevalue(m_ϕ), transpose(u)
end
    
    

function CE_parallel(Φ,prior)
    
    P = ones(S,Z)/Z
    for iter=1:maxiter

        cumsumP = cumsum(P,2)
        
        vec_tile = rand(Uniform(0,1), S * N,1)
        cumsumP_tile = repmat(cumsumP,N)
        which_zone = floor.(findmax(cumsumP_tile .> vec_tile,2)[2] / (S*N) - epsilon) + 1
        u = abs.(repmat(transpose(1:Z),S*N,1) .- which_zone) .< epsilon
        u = transpose(u)
        
        arr_obj = []
        arr_assign = zeros(S,Z,N)
#         tic()
        retobj = @DArray [solve_alloc(u[:,(i-1)*S+1:i*S],Φ,prior) for i = 1:N]
#         toc()
        for i = 1:N
            push!(arr_obj, retobj[i][1])
            arr_assign[:,:,i] = retobj[i][2]
        end
 
        best_ind = sortperm(arr_obj)[1:Threshold]
        P = squeeze(sum(arr_assign[:,:,best_ind],3)/Threshold,3)
    end
    
    u = transpose(P)
    m_ϕ = Model(solver = IpoptSolver(print_level=0))

    @variable(m_ϕ,ϕvar[1:Ncell,1:Z,1:S])
    @variable(m_ϕ,exponential[1:Ncell,1:Z])
    @variable(m_ϕ, aux[1:Ncell,1:Z])
    @objective(m_ϕ, Min, sum(prior .* exponential))
    @NLconstraint(m_ϕ, [i=1:Ncell,j=1:Z], exp(aux[i,j]) <= exponential[i,j])
    @constraint(m_ϕ,aux .== squeeze(-sum(W .* ϕvar .* reshape(u,1,Z,S),3),3))
    @constraint(m_ϕ, exponential .>= 0)
    @constraint(m_ϕ,ϕvar .>= 0)
    @constraint(m_ϕ,[i=1:Z,j=1:S],sum(ϕvar[:,i,j]) <= Φ)

    solve(m_ϕ)
    
    getobjectivevalue(m_ϕ), P, exp.(getvalue(aux)), getvalue(ϕvar)
    
    
    
end


CE_parallel (generic function with 1 method)

In [141]:
tic()
Φ = 5
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,prior)
toc()
print(obj)


elapsed time: 15.884161344 seconds
0.817177375826391

In [138]:
#######################################
# Perturb prior
#######################################
function prior_perturb(mag)
    new_prior = zeros(120)
    counter = 0
    for i = 1:length(new_prior)
        val = reshape(prior,120)[i]
        if val > 0
            perturb = rand(Normal(0,val)) * mag
            newval = val + perturb
            if newval < 0
                counter = counter + 1
            end
            newval = abs(newval)
        else
            newval = val
        end
        new_prior[i] = newval
    end
    new_prior = reshape(new_prior/sum(new_prior),30,4)
    return new_prior, counter
end

prior_perturb (generic function with 1 method)

In [167]:
# small perturbations

num_trials = 1000
nondetection_prior_perturb = zeros(num_trials)
diff_record = 0
counter_record = 0
for i=1:num_trials
    new_prior,counter = prior_perturb(0.1)
    counter_record = counter_record + counter
    diff_record = diff_record + sum(abs.(new_prior - prior))
    nondetection_prior_perturb[i] = sum(new_prior .* exp.( squeeze(-sum(W .* res_alloc .* reshape(transpose(assignment),1,Z,S),3),3)))
end
print(diff_record/num_trials,"\n")
print(counter_record/num_trials,"\n")
summarystats(nondetection_prior_perturb)

0.07958612824628607
0.0


Summary Stats:
Mean:           0.817268
Minimum:        0.812958
1st Quartile:   0.816393
Median:         0.817306
3rd Quartile:   0.818143
Maximum:        0.821726


In [168]:
# medium perturbations

num_trials = 1000
nondetection_prior_perturb = zeros(num_trials)
diff_record = 0
counter_record = 0
for i=1:num_trials
    new_prior,counter = prior_perturb(0.5)
    counter_record = counter_record + counter
    diff_record = diff_record + sum(abs.(new_prior - prior))
    nondetection_prior_perturb[i] = sum(new_prior .* exp.( squeeze(-sum(W .* res_alloc .* reshape(transpose(assignment),1,Z,S),3),3)))
end
print(diff_record/num_trials,"\n")
print(counter_record/num_trials,"\n")
summarystats(nondetection_prior_perturb)

0.3845682268069441
2.713


Summary Stats:
Mean:           0.817358
Minimum:        0.798501
1st Quartile:   0.812826
Median:         0.817192
3rd Quartile:   0.821845
Maximum:        0.837021


In [169]:
# large perturbations

num_trials = 1000
nondetection_prior_perturb = zeros(num_trials)
diff_record = 0
counter_record = 0
for i=1:num_trials
    new_prior,counter = prior_perturb(1)
    counter_record = counter_record + counter
    diff_record = diff_record + sum(abs.(new_prior - prior))
    nondetection_prior_perturb[i] = sum(new_prior .* exp.( squeeze(-sum(W .* res_alloc .* reshape(transpose(assignment),1,Z,S),3),3)))
end
print(diff_record/num_trials,"\n")
print(counter_record/num_trials,"\n")
summarystats(nondetection_prior_perturb)

0.5556839830949762
18.237


Summary Stats:
Mean:           0.817786
Minimum:        0.793592
1st Quartile:   0.811883
Median:         0.817624
3rd Quartile:   0.823936
Maximum:        0.848758


In [179]:
#######################################
# Perturb sensor visibility
#######################################


# data frpm Thi's paper. table 1
sen_visibility_tbl = [0.4 0.5 0.6 0.8 0.5 0.1;
    0.9 0.1 0.1 0.1 0.3 0.5;
    0.3 0.1 0.4 0.6 0.5 0.2;
    0.2 0.7 0.8 0.2 0.4 0.6;
    0.1 0.6 0.7 0.1 0.3 0.5;
    0.8 0.9 0.1 0.7 0.6 0.2]

function visibility_perturb(mag)
    counter_neg = 0
    counter_1plus = 0
    @everywhere new_visibility_tbl = zeros(6,6)
    for i=1:6
        for j=1:6
            val = sen_visibility_tbl[i,j]
            if val > 0
                perturb = rand(Normal(0,val)) * mag
                newval = val + perturb
                if newval < 0
                    counter_neg = counter_neg + 1
                    newval = abs(newval)
                elseif newval > 1
                    counter_1plus = counter_1plus + 1
                    newval = 1
                end
                
            else
                newval = val
            end
            new_visibility_tbl[i,j] = newval
        end
    end
    
    @everywhere s1 = create_pixel_mat(landscape1, new_visibility_tbl[:,1])
    @everywhere s2 = create_pixel_mat(landscape1, new_visibility_tbl[:,2])
    @everywhere s3 = create_pixel_mat(landscape1, new_visibility_tbl[:,3])
    @everywhere s4 = create_pixel_mat(landscape1, new_visibility_tbl[:,4])
    @everywhere s5 = create_pixel_mat(landscape1, new_visibility_tbl[:,5])
    @everywhere s6 = create_pixel_mat(landscape1, new_visibility_tbl[:,6])
    @everywhere new_W = reshape(hcat(s1,s2,s3,s4,s5,s6),Ncell,Z,S) # visibility tensor

    
    return new_W, counter_neg, counter_1plus
end




visibility_perturb (generic function with 1 method)

In [193]:
# small perturbations

num_trials = 1000
nondetection_W_perturb = zeros(num_trials)
diff_record = 0
counter_neg_record = 0
counter_1plus_record = 0
for i=1:num_trials
    new_W, counter_neg, counter_1plus = visibility_perturb(0.1)
    counter_neg_record = counter_neg_record + counter_neg
    counter_1plus_record = counter_1plus_record + counter_1plus
    diff_record = diff_record + sum(abs.(new_W - W))
    nondetection_W_perturb[i] = sum(prior .* exp.( squeeze(-sum(new_W .* res_alloc .* reshape(transpose(assignment),1,Z,S),3),3)))
end
print(diff_record/num_trials,"\n")
print(counter_neg_record/num_trials,"\n")
print(counter_1plus_record/num_trials,"\n")
summarystats(nondetection_W_perturb)

23.844620059098865
0.0
0.298


Summary Stats:
Mean:           0.817676
Minimum:        0.797297
1st Quartile:   0.814003
Median:         0.817373
3rd Quartile:   0.821255
Maximum:        0.834137


In [195]:
# Medium perturbations

num_trials = 1000
nondetection_W_perturb = zeros(num_trials)
diff_record = 0
counter_neg_record = 0
counter_1plus_record = 0
for i=1:num_trials
    new_W, counter_neg, counter_1plus = visibility_perturb(0.5)
    counter_neg_record = counter_neg_record + counter_neg
    counter_1plus_record = counter_1plus_record + counter_1plus
    diff_record = diff_record + sum(abs.(new_W - W))
    nondetection_W_perturb[i] = sum(prior .* exp.( squeeze(-sum(new_W .* res_alloc .* reshape(transpose(assignment),1,Z,S),3),3)))
end
print(diff_record/num_trials,"\n")
print(counter_neg_record/num_trials,"\n")
print(counter_1plus_record/num_trials,"\n")
summarystats(nondetection_W_perturb)

106.54280083812192
0.866
2.921


Summary Stats:
Mean:           0.827099
Minimum:        0.766379
1st Quartile:   0.810588
Median:         0.825275
3rd Quartile:   0.843081
Maximum:        0.895959


In [196]:
# Large perturbations

num_trials = 1000
nondetection_W_perturb = zeros(num_trials)
diff_record = 0
counter_neg_record = 0
counter_1plus_record = 0
for i=1:num_trials
    new_W, counter_neg, counter_1plus = visibility_perturb(1)
    counter_neg_record = counter_neg_record + counter_neg
    counter_1plus_record = counter_1plus_record + counter_1plus
    diff_record = diff_record + sum(abs.(new_W - W))
    nondetection_W_perturb[i] = sum(prior .* exp.( squeeze(-sum(new_W .* res_alloc .* reshape(transpose(assignment),1,Z,S),3),3)))
end
print(diff_record/num_trials,"\n")
print(counter_neg_record/num_trials,"\n")
print(counter_1plus_record/num_trials,"\n")
summarystats(nondetection_W_perturb)

151.78895144138255
5.761
5.423


Summary Stats:
Mean:           0.825574
Minimum:        0.735982
1st Quartile:   0.803558
Median:         0.825958
3rd Quartile:   0.845504
Maximum:        0.916386


In [ ]:
####################################################
## Varying the total resource each sensor has. Not relevant to the discussion of this notebook
#####################################################

Φ_vec = 5:5:40
Φ_vec_len = length(Φ_vec)
obj_vec = zeros(Φ_vec_len,1)

tic()
################## Takes a while to run (~3 mins) ########################
for i=1:Φ_vec_len
    Φ = Φ_vec[i]
    obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,prior)
    obj_vec[i] = obj
    print("Φ: ", Φ, " done.\n")
end
toc()

display(obj_vec)


Amount of resource    Paper        Mine
5                     0.818577     0.816912
10                    0.684701     0.683561
15                    0.580673     0.575656
20                    0.500550     0.495109
25                    0.428900     0.433506
30                    0.372219     0.359684
35                    0.327574     0.303628
40                    0.292742     0.283595

In [ ]:
##############################################
## Moving Target
##############################################

In [197]:
moving_mat= [0.0 0.0 0.0;
             0.0 0.3 0.1;
             0.0 0.1 0.5]
sum(moving_mat)

1.0

In [291]:
# construct a 30x4x30x4 one step moving probability tensor
# first, construct 120x120 one step transition matrix

function construct_c2c(moving_mat)
    # here we number all cells from 1 to 120 in a row-major fashion
    c2c = zeros(Ncell * Z, Ncell * Z)
    for i = 1:(Ncell * Z)
        # Stay in current cell
        c2c[i,i] = moving_mat[2,2]

        # move right
        if mod(i,12) ==  0 # right end 
            c2c[i,i] = c2c[i,i] + moving_mat[2,3]
        else
            c2c[i,i+1] = moving_mat[2,3]
        end

        # move down
        if i > 12 * (10-1) # bottom end
            c2c[i,i] = c2c[i,i] + moving_mat[3,2]
        else
            c2c[i,i+12] = moving_mat[3,2]
        end

        # move diagonal
        if (mod(i,12) == 0) | (i > 12 * (10-1))
            c2c[i,i] = c2c[i,i] + moving_mat[3,3]
        else
            c2c[i,i+12+1] = moving_mat[3,3]
        end


    end

    # convert the 120x120 matrix into 30x4x30x4 tensor
    # Note that Julia is column major: suppose the first (leftmost, upmost) cell
    # is cell 1, cell 2 refers to the cell below cell 1 not the cell to the right
    # This affects how cell index is assigned
    c2c_tensor = zeros(Ncell,Z,Ncell,Z)
    for i=1:Ncell*Z
        zone_prev = (i / 60 > 1) * 2 + ((mod(i,12) / 6 > 1) | (mod(i,12) == 0)) + 1
        offset_prev = [0,6,60,66][zone_prev]
        cell_prev = (rem(i-offset_prev,12) - 1) * 5 + div(i-offset_prev,12) + 1 # column major in julia
        for j=1:Ncell*Z
            zone_cur = (j / 60 > 1) * 2 + ((mod(j,12) / 6 > 1) | (mod(j,12) == 0)) + 1
            offset_cur = [0,6,60,66][zone_cur]
            cell_cur = (rem(j-offset_cur,12) - 1) * 5 + div(j-offset_cur,12) + 1 # column major in julia
            c2c_tensor[cell_prev,zone_prev,cell_cur,zone_cur] = c2c[i,j]
        end
    end
    
    return c2c_tensor
end

construct_c2c (generic function with 1 method)

In [292]:
c2c_tensor = construct_c2c(moving_mat)

30×4×30×4 Array{Float64,4}:
[:, :, 1, 1] =
 0.3  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 ⋮                 
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

[:, :, 2, 1] =
 0.1  0.0  0.0  0.0
 0.3  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 ⋮                 
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 

In [ ]:
# Myopic and Optimal Search Plan (without cross-cueing)

In [262]:
T = 4
Φ = 5
Total_Iter = 4 # the number of iterations to go from myopic search plan to optimal search plan

4

In [263]:
################## Takes a while to run ########################

function mov_target_CE()
    DTensor = ones(Ncell,Z,T)
    moving_target_obj_vec = zeros(Total_Iter,T)
    prob_nondetect_Tensor = zeros(Ncell,Z,T)
    final_assignment = zeros(S,Z,T)
    final_res_alloc = zeros(Ncell,Z,S,T)
    
    
    for i = 1:Total_Iter
        # Initialize UTensor
        UTensor = zeros(Ncell,Z,T)
        UTensor[:,:,1] =  prior


        for τ = 1:4
            if τ > 1
                UTensor[:,:,τ] = sum(sum(c2c_tensor .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
            end
            β = UTensor[:,:,τ] .* DTensor[:,:,τ]
            obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,β)
            final_assignment[:,:,τ] = assignment
            final_res_alloc[:,:,:,τ] = res_alloc
            prob_nondetect_Tensor[:,:,τ] = prob_nondetect
            moving_target_obj_vec[i,τ] = obj
            print("Iter: ",i,". Completed timestep: ",τ,". Obj: ",obj,"\n")
        end

        # Compute the DTensors using the nondetection probabilities at each timestep as calculated
        # in the previous iteration
        for τ = T-1:-1:1
            DTensor[:,:,τ] = sum(sum(c2c_tensor .* 
                    reshape(prob_nondetect_Tensor[:,:,τ+1],1,1,Ncell,Z) .* 
                    reshape(DTensor[:,:,τ+1],1,1,Ncell,Z),3),4)
        end
    end
    
    return moving_target_obj_vec, prob_nondetect_Tensor, final_assignment, final_res_alloc

end

mov_target_CE (generic function with 1 method)

In [264]:
tic()
moving_target_obj_vec, prob_nondetect_Tensor, final_assignment, final_res_alloc = mov_target_CE()
toc()


Iter: 1. Completed timestep: 1. Obj: 0.817736062033825
Iter: 1. Completed timestep: 2. Obj: 0.651539509797221
Iter: 1. Completed timestep: 3. Obj: 0.5035867563133201
Iter: 1. Completed timestep: 4. Obj: 0.38716288211188987
Iter: 2. Completed timestep: 1. Obj: 0.3665433002232844
Iter: 2. Completed timestep: 2. Obj: 0.358335603476455
Iter: 2. Completed timestep: 3. Obj: 0.35705052997208175
Iter: 2. Completed timestep: 4. Obj: 0.3589469987574071
Iter: 3. Completed timestep: 1. Obj: 0.35814163168100965
Iter: 3. Completed timestep: 2. Obj: 0.3532775661697464
Iter: 3. Completed timestep: 3. Obj: 0.3527460241250162
Iter: 3. Completed timestep: 4. Obj: 0.35217503562321456
Iter: 4. Completed timestep: 1. Obj: 0.3515425524100504
Iter: 4. Completed timestep: 2. Obj: 0.3534050210833156
Iter: 4. Completed timestep: 3. Obj: 0.35280325291435555
Iter: 4. Completed timestep: 4. Obj: 0.3525696460593851
elapsed time: 223.382315929 seconds


223.382315929

In [271]:
function compute_non_detection(prior, W, c2c_tensor)
    # Compute nondetection probability at each time step (used in computing U and D)
    nondetection_prob_holder = zeros(Ncell,Z,T)
    for i=1:T
        res_alloc_cur = final_res_alloc[:,:,:,i]
        assign_cur = final_assignment[:,:,i]
        nondetection_prob_holder[:,:,i] = exp.( squeeze(-sum(W .* res_alloc_cur .* reshape(transpose(assign_cur),1,Z,S),3),3))
    end


    # Compute U and D

    UTensor_holder = zeros(Ncell,Z,T)
    UTensor_holder[:,:,1] =  prior
    for i=2:T

        UTensor_holder[:,:,i] = sum(sum(c2c_tensor .* 
            reshape(nondetection_prob_holder[:,:,i-1],Ncell,Z,1,1) .* 
            reshape(UTensor_holder[:,:,i-1],Ncell,Z,1,1),1),2)

    end

    DTensor_holder = ones(Ncell,Z,T)
    for i = T-1:-1:1
        DTensor_holder[:,:,i] = sum(sum(c2c_tensor .* 
                reshape(nondetection_prob_holder[:,:,i+1],1,1,Ncell,Z) .* 
                reshape(DTensor_holder[:,:,i+1],1,1,Ncell,Z),3),4)
    end
    
    β_holder = UTensor_holder .* DTensor_holder
    time_step = T
    return sum(β_holder[:,:,time_step] .* nondetection_prob_holder[:,:,time_step])
end 
     
    


compute_non_detection (generic function with 1 method)

In [272]:
## non detection probaility without perturbation
compute_non_detection(prior, W, c2c_tensor)

0.35256935068369993

In [ ]:
#######################################
# Perturb prior
#######################################


In [274]:
# small perturbations

num_trials = 1000
nondetection_prior_perturb = zeros(num_trials)
diff_record = 0
counter_record = 0
for i=1:num_trials
    new_prior,counter = prior_perturb(0.1)
    counter_record = counter_record + counter
    diff_record = diff_record + sum(abs.(new_prior - prior))
    nondetection_prior_perturb[i] = compute_non_detection(new_prior, W, c2c_tensor)
end
print(diff_record/num_trials,"\n")
print(counter_record/num_trials,"\n")
summarystats(nondetection_prior_perturb)

0.07928416802505962
0.0


Summary Stats:
Mean:           0.352631
Minimum:        0.347914
1st Quartile:   0.351707
Median:         0.352627
3rd Quartile:   0.353568
Maximum:        0.357193


In [275]:
# Med perturbations

num_trials = 1000
nondetection_prior_perturb = zeros(num_trials)
diff_record = 0
counter_record = 0
for i=1:num_trials
    new_prior,counter = prior_perturb(0.5)
    counter_record = counter_record + counter
    diff_record = diff_record + sum(abs.(new_prior - prior))
    nondetection_prior_perturb[i] = compute_non_detection(new_prior, W, c2c_tensor)
end
print(diff_record/num_trials,"\n")
print(counter_record/num_trials,"\n")
summarystats(nondetection_prior_perturb)

0.3860117871352267
2.573


Summary Stats:
Mean:           0.352428
Minimum:        0.329693
1st Quartile:   0.347835
Median:         0.352337
3rd Quartile:   0.357010
Maximum:        0.373791


In [276]:
# Large perturbations

num_trials = 1000
nondetection_prior_perturb = zeros(num_trials)
diff_record = 0
counter_record = 0
for i=1:num_trials
    new_prior,counter = prior_perturb(1)
    counter_record = counter_record + counter
    diff_record = diff_record + sum(abs.(new_prior - prior))
    nondetection_prior_perturb[i] = compute_non_detection(new_prior, W, c2c_tensor)
end
print(diff_record/num_trials,"\n")
print(counter_record/num_trials,"\n")
summarystats(nondetection_prior_perturb)

0.5584690768929785
18.3


Summary Stats:
Mean:           0.353123
Minimum:        0.322453
1st Quartile:   0.346532
Median:         0.353339
3rd Quartile:   0.359827
Maximum:        0.388210


In [ ]:
#######################################
# Perturb sensor visibility
#######################################

In [278]:
# small perturbations

num_trials = 1000
nondetection_W_perturb = zeros(num_trials)
diff_record = 0
counter_neg_record = 0
counter_1plus_record = 0
for i=1:num_trials
    new_W, counter_neg, counter_1plus = visibility_perturb(0.1)
    counter_neg_record = counter_neg_record + counter_neg
    counter_1plus_record = counter_1plus_record + counter_1plus
    diff_record = diff_record + sum(abs.(new_W - W))
    nondetection_W_perturb[i] = compute_non_detection(prior, new_W, c2c_tensor)
end
print(diff_record/num_trials,"\n")
print(counter_neg_record/num_trials,"\n")
print(counter_1plus_record/num_trials,"\n")
summarystats(nondetection_W_perturb)

23.897216763647364
0.0
0.273


Summary Stats:
Mean:           0.353903
Minimum:        0.325120
1st Quartile:   0.347657
Median:         0.353735
3rd Quartile:   0.360160
Maximum:        0.385896


In [279]:
# Med perturbations

num_trials = 1000
nondetection_W_perturb = zeros(num_trials)
diff_record = 0
counter_neg_record = 0
counter_1plus_record = 0
for i=1:num_trials
    new_W, counter_neg, counter_1plus = visibility_perturb(0.5)
    counter_neg_record = counter_neg_record + counter_neg
    counter_1plus_record = counter_1plus_record + counter_1plus
    diff_record = diff_record + sum(abs.(new_W - W))
    nondetection_W_perturb[i] = compute_non_detection(prior, new_W, c2c_tensor)
end
print(diff_record/num_trials,"\n")
print(counter_neg_record/num_trials,"\n")
print(counter_1plus_record/num_trials,"\n")
summarystats(nondetection_W_perturb)

105.33124498429943
0.843
3.062


Summary Stats:
Mean:           0.383029
Minimum:        0.255841
1st Quartile:   0.346973
Median:         0.379490
3rd Quartile:   0.414168
Maximum:        0.625405


In [280]:
# Large perturbations

num_trials = 1000
nondetection_W_perturb = zeros(num_trials)
diff_record = 0
counter_neg_record = 0
counter_1plus_record = 0
for i=1:num_trials
    new_W, counter_neg, counter_1plus = visibility_perturb(1)
    counter_neg_record = counter_neg_record + counter_neg
    counter_1plus_record = counter_1plus_record + counter_1plus
    diff_record = diff_record + sum(abs.(new_W - W))
    nondetection_W_perturb[i] = compute_non_detection(prior, new_W, c2c_tensor)
end
print(diff_record/num_trials,"\n")
print(counter_neg_record/num_trials,"\n")
print(counter_1plus_record/num_trials,"\n")
summarystats(nondetection_W_perturb)

151.45938728075524
5.703
5.425


Summary Stats:
Mean:           0.394578
Minimum:        0.261822
1st Quartile:   0.343975
Median:         0.387776
3rd Quartile:   0.437191
Maximum:        0.620931


In [ ]:
#######################################
# Perturb movement distribution
#######################################

In [283]:
function move_perturb(mag)
    new_move = zeros(9)
    counter = 0
    for i = 1:length(new_move)
        val = reshape(moving_mat,9)[i]
        if val > 0
            perturb = rand(Normal(0,val)) * mag
            newval = val + perturb
            if newval < 0
                counter = counter + 1
            end
            newval = abs(newval)
        else
            newval = val # val == 0 case, do not apply perturbation
        end
        new_move[i] = newval
    end
    new_move = reshape(new_move/sum(new_move),3,3)
    return new_move, counter
end

move_perturb (generic function with 1 method)

In [297]:
# small perturbations

num_trials = 1000
nondetection_move_perturb = zeros(num_trials)
diff_record = 0
counter_record = 0
for i=1:num_trials
    new_move, counter = move_perturb(0.1)
    new_c2c_tensor = construct_c2c(new_move)
    counter_record = counter_record + counter
    diff_record = diff_record + sum(abs.(new_move - moving_mat))
    nondetection_W_perturb[i] = compute_non_detection(prior, W, new_c2c_tensor)
end
print(diff_record/num_trials,"\n")
print(counter_record/num_trials,"\n")
summarystats(nondetection_W_perturb)

0.06213856376701228
0.0


Summary Stats:
Mean:           0.352560
Minimum:        0.330627
1st Quartile:   0.347665
Median:         0.352354
3rd Quartile:   0.357615
Maximum:        0.375724


In [298]:
# Med perturbations

num_trials = 1000
nondetection_move_perturb = zeros(num_trials)
diff_record = 0
counter_record = 0
for i=1:num_trials
    new_move, counter = move_perturb(0.5)
    new_c2c_tensor = construct_c2c(new_move)
    counter_record = counter_record + counter
    diff_record = diff_record + sum(abs.(new_move - moving_mat))
    nondetection_W_perturb[i] = compute_non_detection(prior, W, new_c2c_tensor)
end
print(diff_record/num_trials,"\n")
print(counter_record/num_trials,"\n")
summarystats(nondetection_W_perturb)

0.3132406956662055
0.084


Summary Stats:
Mean:           0.357542
Minimum:        0.282003
1st Quartile:   0.330958
Median:         0.353074
3rd Quartile:   0.378474
Maximum:        0.520167


In [299]:
# Large perturbations

num_trials = 1000
nondetection_move_perturb = zeros(num_trials)
diff_record = 0
counter_record = 0
for i=1:num_trials
    new_move, counter = move_perturb(1)
    new_c2c_tensor = construct_c2c(new_move)
    counter_record = counter_record + counter
    diff_record = diff_record + sum(abs.(new_move - moving_mat))
    nondetection_W_perturb[i] = compute_non_detection(prior, W, new_c2c_tensor)
end
print(diff_record/num_trials,"\n")
print(counter_record/num_trials,"\n")
summarystats(nondetection_W_perturb)

0.4692413238667059
0.62


Summary Stats:
Mean:           0.364373
Minimum:        0.270889
1st Quartile:   0.323145
Median:         0.353290
3rd Quartile:   0.396531
Maximum:        0.531975


In [ ]:
####################################################
## Comparing my own implementation results vs the paper. Not relevant to the discussion of this notebook
#####################################################

# rows denote each iteration of going to myopic search plan to
# optimal search plan
# column denote each time step within the iteration
moving_target_obj_vec


Time    Myopic                  Optimal
        Paper       Mine        Paper       Mine
1       0.8198461   0.817348    0.2962617   0.352191
2       0.6379034   0.652625    0.2958389   0.351842
3       0.4727772   0.506620    0.2954212   0.351133
4       0.3325033   0.392449    0.2950101   0.350590